# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
display(df.head())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
df = df.astype({'calls': 'int', 'messages': 'int'})

In [6]:
df.duplicated().sum()

0

In [5]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

**Общая информация**
1. is_ultra - целевой признак.
2. Остальные столбцы - признаки.
3. Предобработка данных не требуется.
4. Задача относится к задаче классификации.
5. 70% пользователей на тарифе smart, 30% - пользуются ultra. Необходима стратификация. 

## Разбейте данные на выборки

In [6]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Соотношение размеров выборок train и test: 80/20  
Соотношение классов: 70/30

In [7]:
rns = 123

In [8]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, 
                                                                          random_state=rns,
                                                                         stratify = target)

Провеярем соотношение классов в каждой выборке:

In [12]:
print(target_train.value_counts(normalize=True))
print(target_test.value_counts(normalize=True))

0    0.693504
1    0.306496
Name: is_ultra, dtype: float64
0    0.693624
1    0.306376
Name: is_ultra, dtype: float64


## Исследуйте модели

### Дерево решений

**Создадим модель с параметрами по умолчанию:**

In [11]:
model_dtc = DecisionTreeClassifier(random_state=rns)
model_dtc.fit(features_train, target_train)
prediction_dtc = model_dtc.predict(features_test)
result_dtc = accuracy_score(target_test,prediction_dtc)
print('accuracy:', result_dtc);

accuracy: 0.7402799377916018


**Подберем гиперпараметры с помощью RandomizedSearchCV:**

In [13]:
criterion = ['entropy', 'gini']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 15, num = 10)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
param_rs_dtc = {'criterion': criterion,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [14]:
rs_dtc = RandomizedSearchCV(model_dtc, 
                        param_rs_dtc, 
                        n_iter = 200, 
                        cv = 4, 
                        verbose = 1, 
                        n_jobs=-1, 
                        random_state=rns)
rs_dtc.fit(features_train, target_train)
rs_dtc.best_params_

Fitting 4 folds for each of 200 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:   10.4s finished


{'min_samples_split': 23,
 'min_samples_leaf': 2,
 'max_depth': 8,
 'criterion': 'gini'}

**Сохраняем лучшую модель**

In [15]:
model_dtc_rs = rs_dtc.best_estimator_

**Изучим лучшие варианты гиперпараметров:**

In [16]:
rs_dtc = pd.DataFrame(rs_dtc.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
rs_dtc = rs_dtc.drop([
            'mean_fit_time', 
            'std_fit_time', 
            'mean_score_time',
            'std_score_time', 
            'params', 
            'split0_test_score', 
            'split1_test_score', 
            'split2_test_score',
            'split3_test_score',
            'std_test_score'],
            axis=1)

In [17]:
rs_dtc.head(5)

,param_min_samples_split,param_min_samples_leaf,param_max_depth,param_criterion,mean_test_score,rank_test_score
0,23,2,8,gini,0.802412,1
1,28,2,8,gini,0.802412,1
2,50,7,7,gini,0.799300,3
3,39,12,7,entropy,0.798911,4
4,18,2,8,gini,0.798522,5


**Используя то, что мы выяснили с помощью RandomizedSearchCV, исследуем значения гиперпараметров, которые лучше всего себя показали:**

In [18]:
criterion = ['gini']
max_depth = [7, 8]
min_samples_split = [23, 28, 50]
min_samples_leaf = [2, 7]
param_gs_dtc = {'criterion': criterion,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [19]:
gs_dtc = GridSearchCV(model_dtc, 
                      param_gs_dtc, 
                      cv = 10, 
                      verbose = 1, 
                      n_jobs=-1)
gs_dtc.fit(features_train, target_train)
gs_dtc.best_params_

[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    1.7s finished


{'criterion': 'gini',
 'max_depth': 7,
 'min_samples_leaf': 7,
 'min_samples_split': 28}

**Сохраняем лучшую модель**

In [20]:
model_dtc_gs = gs_dtc.best_estimator_

**Сравним accuracy до и после подбора гиперпараметров:**

In [72]:
print('accuracy:', model_dtc.score(features_test, target_test))
print('accuracy:', model_dtc_rs.score(features_test, target_test))
print('accuracy:', model_dtc_gs.score(features_test, target_test))

accuracy: 0.7402799377916018
accuracy: 0.7776049766718507
accuracy: 0.7853810264385692


**Вывод**  
Модель DTC показала наилучший результат 0.78 после подбора параметров с помощью GridSearchCV.

### Случайный лес

**Создадим модель с параметрами по умолчанию:**

In [22]:
model_rf = RandomForestClassifier(random_state=rns)
model_rf.fit(features_train, target_train)
prediction_rf = model_rf.predict(features_test)
result_rf = accuracy_score(target_test,prediction_rf)
print('accuracy:', result_rf);

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


accuracy: 0.7884914463452566


**Подберем гиперпараметры с помощью RandomizedSearchCV:**

In [29]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 15, num = 15)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
bootstrap = [True, False]
param_rs_rf = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [23]:
rs_rf = RandomizedSearchCV(model_rf, 
                        param_rs_rf, 
                        n_iter = 100, 
                        cv = 4, 
                        verbose = 1, 
                        n_jobs=-1, 
                        random_state=rns)
rs_rf.fit(features_train, target_train)
rs_rf.best_params_

Fitting 4 folds for each of 100 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed: 14.9min finished


{'n_estimators': 200,
 'min_samples_split': 50,
 'min_samples_leaf': 2,
 'max_features': 'log2',
 'max_depth': 13,
 'bootstrap': False}

**Сохраняем лучшую модель**

In [26]:
model_rf_rs = rs_rf.best_estimator_

**Изучим лучшие варианты гиперпараметров:**

In [27]:
rs_rf = pd.DataFrame(rs_rf.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
rs_rf = rs_rf.drop([
            'mean_fit_time', 
            'std_fit_time', 
            'mean_score_time',
            'std_score_time', 
            'params', 
            'split0_test_score', 
            'split1_test_score', 
            'split2_test_score',
            'split3_test_score',
            'std_test_score'],
            axis=1)

In [28]:
rs_rf.head(5)

,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_features,param_max_depth,param_bootstrap,mean_test_score,rank_test_score
0,200,50,2,log2,13,False,0.813691,1
1,400,2,12,log2,9,False,0.811357,2
2,100,18,2,log2,15,True,0.810968,3
3,300,2,7,log2,11,False,0.810580,4
4,700,34,7,sqrt,12,True,0.810580,4


**Используя то, что мы выяснили с помощью RandomizedSearchCV, исследуем значения гиперпараметров, которые лучше всего себя показали:**

In [40]:
n_estimators = [100, 200, 400]
max_features = ['log2']
max_depth = [9, 13]
min_samples_split = [18, 50]
min_samples_leaf = [12]
bootstrap = [True, False]
param_gs_rf = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [41]:
gs_rf = GridSearchCV(model_rf, 
                      param_gs_rf, 
                      cv = 10, 
                      verbose = 1, 
                      n_jobs=-1)
gs_rf.fit(features_train, target_train)
gs_rf.best_params_

Fitting 10 folds for each of 24 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  5.3min finished


{'bootstrap': True,
 'max_depth': 13,
 'max_features': 'log2',
 'min_samples_leaf': 12,
 'min_samples_split': 50,
 'n_estimators': 100}

**Сохраняем лучшую модель**

In [42]:
model_rf_gs = gs_rf.best_estimator_

**Сравним accuracy до и после подбора гиперпараметров:**

In [43]:
print('accuracy:', model_rf.score(features_test, target_test))
print('accuracy:', model_rf_rs.score(features_test, target_test))
print('accuracy:', model_rf_gs.score(features_test, target_test))

accuracy: 0.7884914463452566
accuracy: 0.8118195956454122
accuracy: 0.8009331259720062


**Вывод**  
Модель RF показала наилучший результат 0.81 после подбора параметров с помощью RandomizedSearchCV.

### Логистическая регрессия

**Отделим от обучающей выборки валидационную:**

In [61]:
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size=0.25, 
                                                                          random_state=rns,
                                                                         stratify = target_train)

**Обучим модель**

In [64]:
model_lr = LogisticRegression(random_state=rns)
model_lr.fit(features_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [70]:
print('Accuracy на валидационной выборке:', model_lr.score(features_valid, target_valid))
print('Accuracy на тестовой выборке:', model_lr.score(features_test, target_test))

Accuracy на валидационной выборке: 0.7511664074650077
Accuracy на тестовой выборке: 0.7682737169517885


### Общий вывод

Проведено исследование 3х моделей: дерево решений, случайный лес и логистическая регрессия. В двух первых моделях сделан подбор различных гиперпараметров и выбраны лучшие модели.  
Наилучший результат показала модель случайный лес с accuracy 0.81. В то же время, обучение этой модели потребовало больше всего времени.
Самый быстрый алгоритм - логистическая регрессия; модель обучилась быстро, в то же время показав хороший результат.

## Проверьте модель на тестовой выборке

In [74]:
print('Accuracy модели случайный лес на тестовой выборке:', model_rf_rs.score(features_test, target_test))

Accuracy модели случайный лес на тестовой выборке: 0.8118195956454122


## (бонус) Проверьте модели на адекватность

In [91]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

69% клиентов пользуются тарифом Smart, в таком случае если бы наша модель всегда предсказывала тариф Smart, доля правильных ответов была бы 69%. Доля правильных ответов лучшей созданной модели равна 81%, можно сделать вывод, что модель адекватна. 